In [153]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

from sklearn.metrics import accuracy_score, precision_score
from sklearn.model_selection import train_test_split

In [154]:
class CaseBasedReasoning:

    def __init__(self, dataset: pd.DataFrame):

        # Salvo il Dataframe
        self.dataset = dataset

        # Creo e addestro il modello K = 6 scelto da un Silhouette Score
        self.model = KMeans(n_clusters=6, n_init='auto', random_state=46)

    def fit(self, X_train: pd.DataFrame):

        # Addestro il modello e metto le etichette nel dataset di train
        self.X_train = X_train
        self.X_train['cluster'] = self.model.fit_predict(self.X_train.values)

    def predict(self, X_test: pd.DataFrame):

        # Prevedo il cluster di appartenenza 
        predizioni_cluster = self.model.predict(X_test.values)
        X_test['cluster'] = predizioni_cluster

        # Valuto l'azione migliore per ogni cluster
        azioni_predette_per_cluster = []
        for cluster in range(6):
            indici_cluster = self.X_train.loc[self.X_train['cluster'] == cluster].index.to_list()

            # Prendo solo gli indici che hanno bmi negativo
            indici_bmi_negativi = []

            for indice in indici_cluster:
                bmi_caso = self.dataset.iloc[indice, -2]
                
                if bmi_caso <= 0:
                    indici_bmi_negativi.append(indice)
            
            # Calcolo l'azione
            azione_predetta = round(self.dataset.iloc[indici_bmi_negativi, -1].mean())
            azioni_predette_per_cluster.append(azione_predetta)

        mappa_elementi = {indice: elemento for indice, elemento in zip(range(len(azioni_predette_per_cluster)), azioni_predette_per_cluster)}
        X_test['y_predict'] = X_test['cluster'].map(mappa_elementi)

        return X_test['y_predict']
            
    def evaluate(self):
        w_match_gain, w_match_loss, w_miss_gain, w_miss_loss = 0, 0, 0, 0

        for idx_caso, variazione_bmi, azione_predetta in self.y_predict:

            azione_effettiva = self.dataset.iloc[int(idx_caso), -1]

            # Verifica validità predizione
            if (azione_predetta == azione_effettiva) and (variazione_bmi > 0):
                w_match_gain += 1
            elif (azione_predetta != azione_effettiva) and (variazione_bmi > 0):
                w_miss_gain += 1
            elif (azione_predetta == azione_effettiva) and (variazione_bmi <= 0):
                w_match_loss += 1
            elif (azione_predetta != azione_effettiva) and (variazione_bmi <= 0):
                w_miss_loss += 1
        
        # Calcolo la precisione del modello
        precisione = w_match_loss / (w_match_loss + w_miss_loss)

        # Calcolo l'accuratezza del modello
        accuratezza = (w_match_gain + w_match_loss) / (w_match_loss + w_match_gain + w_miss_loss + w_miss_gain)

        return precisione, accuratezza

In [155]:
def main():
    file_path = '../../dataset/dataset_cbr.csv'
    df = pd.read_csv(file_path)

    # Divido il Dataset in Train e Test
    X_train, X_test, _, y_test = train_test_split(df, df.iloc[:, -1], test_size=0.3, random_state=46)

    model = CaseBasedReasoning(df)
    model.fit(X_train)
    y_predict = model.predict(X_test)

    acc = accuracy_score(y_test, y_predict)
    pr = precision_score(y_test, y_predict, average='weighted')

    display('Accuratezza: {:.2f}'.format(acc))
    display('Precisione: {:.2f}'.format(pr))
    

In [156]:
main()

c:\Users\57g-74a3\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'Accuratezza: 0.30'

'Precisione: 0.19'